In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np

In [15]:
# need to define following things
# x: observed context
# w in W: decision to be made
# c: parameter of cost/utility function
# f(c, w): cost/utility function

# formulation: min_{w in W} f(c, w) 
# where {c in U(x)}

# (x) 1. generate x
# (x) 2. generate c = g(x)
# (x) 3. choose a function f(c, w)
# (*) 4. determine how to solve min_{w in W} f(c, w) for some simple choices of W: how to solve?
# ( ) 5. produce a dataset {(c, x)}
# ( ) 6. learn a predictor g^(x) = c
# ( ) 7. conformalize g^(x) to produce C(x) regions
# ( ) 8. solve min_{w in C(x)} f(c, w)

# goal: determine allocation of items to buy (where utility of each is unknown)
n = 20 # number of items
d = 10 # dim of context to utility

In [46]:
def g(x):
    theta = np.random.randint(low=0, high=2, size=(n, d))
    c = (theta @ x) ** 2
    return c, (c * 4/5, c * 6/5) # lower, upper bounds for c

x = np.random.uniform(low=0, high=4, size=(d)) # contextual observation
c_mid, (c_lb, c_ub) = g(x)

In [47]:
# w in [0, 1]^n such that p^T w < B
p = np.random.randint(low=0, high=1000, size=n)
u = np.random.uniform(low=0, high=1)
B = np.random.uniform(np.max(p), np.sum(p) - u * np.max(p))

In [50]:
model = ro.Model()

w = model.dvar(n)
c = model.rvar(n)
uset = (c_lb <= c, c <= c_ub)

model.minmax(-c @ w, uset)
# model.min(-c_mid @ w)
model.st(w <= 1)
model.st(w >= 0)
model.st(p @ w <= B)

model.solve(grb)
rsome_result = w.get()

Being solved by Gurobi...


Solution status: 2
Running time: 0.0006s


In [51]:
model_val = model.get()

-544.3762849387068